In [97]:
import numpy as np
# import matplotlib.pyplot as plt
import plotly.graph_objects as go

from boat_simulation import speed_interp, wrap_phase

from boat_simulation import angle_difference as angle_difference_simulation

BOAT_LENGTH = 7
TACK_PENALTY_SECONDS = 15
DEFAULT_DT = 10  # Time step
SOG_max = 10  # Assuming a maximum SOG for the heuristic calculation
DEAD_ZONE = 30 * np.pi/180  # Dead zone for tacking
GRID_RESOLUTION = 4 # grid resolution in meters

class Node:
    def __init__(self, position, heading, speed, parent=None, tacking=False):
        self.position = position
        self.parent = parent
        self.g = 0  # Cost from start to current node
        self.h = 0  # Heuristic - estimated cost from current node to end
        self.f = 0  # Total cost
        self.tacking = tacking  # Indicates if this node involved a tack
        self.heading = heading
        self.speed = speed

    def __eq__(self, other):
        return np.all(self.position == other.position) and self.heading == other.heading and self.tacking == other.tacking and self.speed == other.speed

def angle_difference(a, b):
    if a is None:
        return b

    if b is None:
        return a

    return angle_difference_simulation(a, b)

def vmg(current_pos, mark_pos, water_current, speed, heading):
    """
    Calculates the speed component towards a mark.

    Parameters:
    - current_pos: Tuple (x, y) representing the current position.
    - mark_pos: Tuple (x, y) representing the mark's position.
    - speed: The current speed.
    - heading: The current heading in degrees.

    Returns:
    - The speed component towards the mark.
    """
    # Convert speed and heading to a velocity vector
    velocity_vector = np.array([speed * np.sin(heading), speed * np.cos(heading)]) + water_current

    # Calculate the direction vector from the current position to the mark
    direction_vector = np.array([mark_pos[0] - current_pos[0], mark_pos[1] - current_pos[1]])

    # Calculate the dot product of the velocity and direction vectors
    dot_product = np.dot(velocity_vector, direction_vector)

    # Calculate the magnitude of the direction vector
    direction_magnitude = np.linalg.norm(direction_vector)

    # Calculate the speed component towards the mark
    speed_towards = dot_product / direction_magnitude

    return speed_towards

def get_water_current(position, currents, xs, ys):
    x_idx = np.argmin(np.abs(position[0] - xs))
    y_idx = np.argmin(np.abs(position[1] - ys))
    return currents[x_idx, y_idx]

def on_segment(px, py, qx, qy, rx, ry):
    """ Check if point R lies on line segment PQ """
    if min(px, qx) <= rx <= max(px, qx) and min(py, qy) <= ry <= max(py, qy):
        return True
    return False

def orientation(px, py, qx, qy, rx, ry):
    """ Return orientation of the triplet (p, q, r).
        0 -> p, q and r are collinear
        1 -> Clockwise
        2 -> Counterclockwise
    """
    val = (qy - py) * (rx - qx) - (qx - px) * (ry - qy)
    if val == 0:
        return 0
    elif val > 0:
        return 1
    else:
        return 2

def segments_intersect(p1x, p1y, q1x, q1y, p2x, p2y, q2x, q2y):
    """ Return True if line segment 'p1q1' and 'p2q2' intersect. """
    o1 = orientation(p1x, p1y, q1x, q1y, p2x, p2y)
    o2 = orientation(p1x, p1y, q1x, q1y, q2x, q2y)
    o3 = orientation(p2x, p2y, q2x, q2y, p1x, p1y)
    o4 = orientation(p2x, p2y, q2x, q2y, q1x, q1y)

    # General case
    if (o1 != o2 and o3 != o4):
        return True

    # Special Cases (checking if the points are collinear and on segment)
    if (o1 == 0 and on_segment(p1x, p1y, q1x, q1y, p2x, p2y)):
        return True
    if (o2 == 0 and on_segment(p1x, p1y, q1x, q1y, q2x, q2y)):
        return True
    if (o3 == 0 and on_segment(p2x, p2y, q2x, q2y, p1x, p1y)):
        return True
    if (o4 == 0 and on_segment(p2x, p2y, q2x, q2y, q1x, q1y)):
        return True

    return False  # Doesn't intersect

def intersects_obstacle(start_pos, end_pos, obstacles):
    lx0, ly0, lx1, ly1 = start_pos[0], start_pos[1], end_pos[0], end_pos[1]
    for (rx0, rx1, ry0, ry1) in obstacles:
        intersects = (
            segments_intersect(lx0, ly0, lx1, ly1, rx0, ry0, rx1, ry0) or
            segments_intersect(lx0, ly0, lx1, ly1, rx0, ry1, rx1, ry1) or
            segments_intersect(lx0, ly0, lx1, ly1, rx0, ry0, rx0, ry1) or
            segments_intersect(lx0, ly0, lx1, ly1, rx1, ry0, rx1, ry1)
        )
        if intersects:
            return True

    return False

def get_neighbors(node, goal, obstacles, water_currents, xs, ys, grid_resolution, dt, min_x, max_x, min_y, max_y):
    angles = [x * np.pi/180 for x in range(40, 180, 1)]
    # angles = np.array([0, 45, 137, 180]) * np.pi/180

    def valid_angle(angle):
        if node.heading is None:
            return True
        diff = np.abs(angle_difference(angle, node.heading))
        return diff <= np.pi / 2

    angles = angles + [2 * np.pi - x for x in angles]
    angles = [x for x in angles if valid_angle(x)]
    if len(angles) == 0:
        return []
    angles = np.array(angles)

    # if node.heading is None:
    #     pass
    # elif node.heading <= DEAD_ZONE or node.heading >= 2 * np.pi - DEAD_ZONE:
    # elif node.heading < np.pi:
    #     angles = 2 * np.pi - angles

    speeds = speed_interp(angles)
    water_current = get_water_current(node.position, water_currents, xs, ys)

    vmgs = [vmg(node.position, goal.position, water_current, speed, angle) for angle, speed in zip(angles, speeds)]

    idx = np.argsort(vmgs)[-min(len(angles), 3):]
    tacking_angles = angles[idx]

    if node.heading:
        angles = np.concatenate([tacking_angles, node.heading.reshape((1,))])
        speeds = np.concatenate([speeds[idx], speeds[np.argmin(np.abs(node.heading - angles)).reshape((1,))]])
    else:
        angles = tacking_angles
        speeds = np.reshape(speeds[idx], (-1,))

    speed_x = np.sin(angles) * speeds + water_current[0]
    speed_y = np.cos(angles) * speeds + water_current[1]

    next_x = node.position[0] + speed_x * dt
    next_y = node.position[1] + speed_y * dt

    next_pos = np.stack([next_x, next_y], axis=1)
    next_pos = np.round(next_pos / grid_resolution) * grid_resolution

    def valid_position(pos, min_x, max_x, min_y, max_y):
        return not intersects_obstacle(node.position, pos, obstacles) and pos[0] >= min_x and pos[0] <= max_x and pos[1] >= min_y and pos[1] <= max_y


    # Calculate new positions based on current node, VMG angle, and boat speed
    speeds = np.sqrt(speed_x ** 2 + speed_y ** 2)
    neighbors = [Node(next_pos[i], angle, speeds[i], node, node.heading != angle) for i, angle in enumerate(angles) if valid_position(next_pos[i], min_x, max_x, min_y, max_y)]

    # Implement the logic to calculate neighbor positions considering tacking and non-tacking scenarios
    return neighbors

def heuristic(node, goal):
    # Euclidean distance to goal divided by max SOG
    return np.linalg.norm(node.position - goal.position)/SOG_max

def goal_reached(node, goal):
    dx = node.position[0] - goal.position[0]
    dy = node.position[1] - goal.position[1]
    return np.sqrt(dx**2 + dy**2) <= 3 * BOAT_LENGTH

def add_to_open(open_list, neighbor):
    for node in open_list:
        if neighbor == node and neighbor.g >= node.g:
            return False
    return True

def a_star(start, goal, obstacles, water_currents, xs, ys, start_heading=None, grid_resolution=GRID_RESOLUTION, dt=DEFAULT_DT, min_x=-400, max_x=400, min_y=-400, max_y=400):
    open_list = []
    closed_list = []

    start_node = Node(start, start_heading, 0, None)
    goal_node = Node(goal, None, 0, None)

    open_list.append(start_node)

    while open_list:
        current_node = min(open_list, key=lambda o: o.f)
        open_list.remove(current_node)
        closed_list.append(current_node)

        # Check if we reached the goal within a certain radius
        if goal_reached(current_node, goal_node):
            path = []
            while current_node is not None:
                path.append((current_node.position[0], current_node.position[1], current_node.heading, current_node.speed, current_node.g))
                current_node = current_node.parent
            return path[::-1]  # Return reversed path

        # Generate children
        for neighbor in get_neighbors(current_node, goal_node, obstacles, water_currents, xs, ys, grid_resolution, dt, min_x, max_x, min_y, max_y):
            if any([n == neighbor for n in closed_list]):
                continue

            # Update the cost considering tacking penalty
            neighbor.g = current_node.g + dt + (TACK_PENALTY_SECONDS if neighbor.tacking else 0)
            neighbor.h = heuristic(neighbor, goal_node)
            neighbor.f = neighbor.g + neighbor.h

            if add_to_open(open_list, neighbor):
                open_list.append(neighbor)

    return None  # No path found

In [101]:
# Define start and goal
start = np.array([0, 0])
goal = np.array([0, 250])

min_x = -300
max_x = 300
min_y = -300
max_y = 300

xs=np.arange(min_x, max_x, GRID_RESOLUTION)
ys=np.arange(min_y, max_y, GRID_RESOLUTION)

water_currents = np.zeros((len(xs), len(ys), 2))
# water_currents[xs > 10, :] = np.array([-10, 10])
# water_currents[xs < -10, :] = np.array([10, 10])

obstacles = [(-100, 50, 75, 125)]

# Run A*
path = a_star(start, goal, obstacles, water_currents, xs, ys, grid_resolution=4, dt=DEFAULT_DT//2)
colormap = ['blue', 'red', 'green', 'orange', 'black']


def plot_path(path, targets, obstacles, water_currents, xs, ys):
    if path:
        # Plotting the path
        x_coords, y_coords, headings, speeds, color_idx = zip(*path)

        headings = [(h or 0) * 180/np.pi if (h or 0) < np.pi else (h or 0) * 180/np.pi - 360 for h in headings]
        fig = go.Figure()

        colors = [colormap[c % len(colormap)] for c in color_idx]
        scatter = go.Scatter(x=x_coords, y=y_coords, mode='lines+markers', name='Path', hoverinfo='text', text=[f'X: {x}; Y: {y}; Heading: {heading}°; Speed: {speed}' for x, y, heading, speed in zip(x_coords, y_coords, headings, speeds)])
        scatter.marker.color = colors
        fig.add_trace(scatter)

        target_xs = [x[0] for x in targets]
        target_ys = [x[1] for x in targets]
        fig.add_trace(go.Scatter(x=target_xs, y=target_ys, mode='markers', name='Goal',
                            marker=dict(color='red', size=12)))

        # Add arrows for water currents
        arrows = []
        for i, x in enumerate(xs):
            if i % 10 != 0:
                continue
            for j, y in enumerate(ys):
                if j % 10 != 0:
                    continue
                u, v = water_currents[i, j]
                if np.linalg.norm([u, v]) > 0:  # Only plot if there is a significant current
                    arrow = go.layout.Annotation(dict(x=x+20*np.sign(u), y=y+20*np.sign(v),
                                    ax=x, ay=y,  # Adjust the scaling factor based on your grid resolution
                                    xref="x", yref="y",
                                    axref="x", ayref="y",
                                    showarrow=True, arrowhead=1, arrowsize=2, arrowwidth=1, opacity=0.25, arrowcolor='green'))

                    arrows.append(arrow)

        rectangles = []
        for (x0, x1, y0, y1) in obstacles:
            rect = go.layout.Shape(dict(type="rect", x0=x0, x1=x1, y0=y0, y1=y1, line=dict(color="grey"), fillcolor="grey", opacity=0.5))
            rectangles.append(rect)

        fig.update_layout(title='Optimal Sailing Path', xaxis_title='X Position (m)', yaxis_title='Y Position (m)',
                    hovermode='closest', width=600, height=600, annotations=arrows, shapes=rectangles)
        fig.update_xaxes(range=[-350, 350])
        fig.update_yaxes(range=[-350, 350])
        fig.show()
    else:
        print("No path found")

if path is not None:
    plot_path([(x, y, z, w, 0) for x, y, z, w, _ in path], [goal], obstacles, water_currents, xs, ys)
else:
    print("No path found")

In [3]:
# Plotting the path
x_coords = np.arange(0, 180, 1)
y_coords = speed_interp(x_coords * np.pi / 180) * -np.cos(x_coords * np.pi / 180)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_coords, y=y_coords, mode='lines+markers', name='Path',
                        hoverinfo='text', hovertext=[f'Angle: {angle}°' for angle in x_coords]))
fig.update_layout(height=800, width=800)
fig.show()

In [4]:
targets = [(0, 250), (250, 250), (250, 0), (0, 0), (-250, 0), (-250, -250), (0, -250), (0, 0)]
# targets = [(0, 250), (250, 250), (250, 0), (0, 0)] # , (-250, 0), (-250, -250), (0, -250), (0, 0)]
# targets = [(0, 250)]

bounds = {'min_x': -300, 'max_x': 300, 'min_y': -300, 'max_y': 300}
targets = [(0, 250, GRID_RESOLUTION, DEFAULT_DT, bounds), (0, -250, GRID_RESOLUTION * 4, DEFAULT_DT * 2, {k: v * 2 for k, v in bounds.items()}), (0, 0, GRID_RESOLUTION, DEFAULT_DT, bounds)]
full_trajectory = []

start_x = 0
start_y = 0
start_heading = 2*np.pi - np.pi/4
start_vmg = 0
previous_target_x = 0
previous_target_y = 0
for mark_idx, (target_x, target_y, grid_resolution, dt, bounds) in enumerate(targets):
    # Define start and goal
    start = np.array([start_x - previous_target_x, start_y - previous_target_y])
    goal = np.array([target_x - previous_target_x, target_y - previous_target_y])

    print(start, goal)
    # Run A*
    path = a_star(start, goal, start_heading=start_heading, grid_resolution=grid_resolution, dt=dt, **bounds)

    print(len(path))

    path = [(x + previous_target_x, y + previous_target_y, heading, mark_idx) for x, y, heading, _ in path]
    start_x, start_y, start_heading, _ = path[-1]
    previous_target_x = target_x
    previous_target_y = target_y

    print('x, y:', start_x, start_y)


    full_trajectory.extend(path)


plot_path(full_trajectory, targets)

[0 0] [  0 250]


TypeError: a_star() missing 3 required positional arguments: 'water_currents', 'xs', and 'ys'